In [97]:
%matplotlib inline
from amb_sdk.sdk import DarwinSdk
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import datetime

In [98]:
#Set darwin url
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

(True, 'https://amb-demo-api.sparkcognition.com/v1/')

In [99]:
#Setting environmental variables 
USERNAME='idunlap@rocketmail.com'
PASSWORD='5uVGHsTHrQ'

#Setting path to datasets

#Here we are using the 17/18 data to train season to train and the 16/17 season data to test
#Since we're looking for predicted position, we use datasets with single-position players (SPP)
#Since we cannot run analyze predict on datasets with more than 500 rows, we are only using the top 500 players 
#from the 16/17 season data in priority of players with the most games played
PATH_TO_DATASET = 'sets/'
TRAIN_DATASET='12-16_Combines_Stats_Per_100_Possessions_SPP.csv'
PREDICT_DATASET = '17_18_Season_Stats_Per_100_Possessions_SPP_limit_100.csv'

#creating a datetime timestamp for later use
TS = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [100]:
#Log in
status, msg = ds.auth_login_user(USERNAME,PASSWORD)
if not status:
    print(msg)
    
#Pre-build cleanup
ds.delete_all_datasets()
ds.delete_all_models()
ds.delete_all_artifacts()

Deleting 12-16_Combines_Stats_Per_100_Possessions_SPP.csv
Deleting model-20190504134832
Deleting model-20190504133249
Deleting b58e8a85f7384aab8846dbf4498f78d9
Error removing artifact "b58e8a85f7384aab8846dbf4498f78d9" - 404: NOT FOUND - {"message": "Failed to find artifact b58e8a85f7384aab8846dbf4498f78d9"}

Deleting 44d13b7470664d8d9635083fbcf5938d
Error removing artifact "44d13b7470664d8d9635083fbcf5938d" - 404: NOT FOUND - {"message": "Failed to find artifact 44d13b7470664d8d9635083fbcf5938d"}

Deleting 2675cf18b2934b3d8457c3c6bc922d54
Error removing artifact "2675cf18b2934b3d8457c3c6bc922d54" - 404: NOT FOUND - {"message": "Failed to find artifact 2675cf18b2934b3d8457c3c6bc922d54"}

Deleting 101dd4ea79be471e9e8e6b933a20b3e6
Error removing artifact "101dd4ea79be471e9e8e6b933a20b3e6" - 404: NOT FOUND - {"message": "Failed to find artifact 101dd4ea79be471e9e8e6b933a20b3e6"}

Deleting 584ac6f6db89442aaac87684a7fabea8
Error removing artifact "584ac6f6db89442aaac87684a7fabea8" - 404: NO

(True, None)

In [101]:
# Preview dataset and display the first 5 entries
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,string_field_29,ORtg,DRtg
0,262,Alex Len\lenal01,C,22,PHO,78,46,1821,7.1,16.7,...,15.9,2.6,1.0,1.7,3.9,6.2,18.8,NaN,96.0,107
1,260,Alex Len\lenal01,C,23,PHO,77,34,1560,7.1,14.2,...,15.6,1.4,1.2,3.0,3.1,7.4,18.8,NaN,108.0,108
2,279,Alex Len\lenal01,C,21,PHO,69,44,1518,5.9,11.6,...,14.9,1.1,1.1,3.4,2.4,7.0,14.2,NaN,107.0,103
3,267,Alex Len\lenal01,C,20,PHO,42,3,362,4.6,10.8,...,13.7,0.6,0.6,2.5,3.6,9.4,11.9,NaN,92.0,106
4,264,Alex Kirk\kirkal01,C,23,CLE,5,0,14,3.7,14.9,...,3.7,3.7,0.0,0.0,0.0,3.7,14.9,NaN,112.0,115


In [102]:
# Upload dataset to darwin
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
print(status)
print(dataset)

if not status:
    print(dataset)

True
{'dataset_name': '12-16_Combines_Stats_Per_100_Possessions_SPP.csv'}


In [103]:
#Analyze data 
status, analyze_id = ds.analyze_data(TRAIN_DATASET, 
                                     job_name = 'Darwin_analyze_data_job' + "-" + TS, 
                                     artifact_name = 'Darwin_analyze_data_artifact' + "-" + TS)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job' + "-" + TS)
else:
    print(analyze_id)

{'status': 'Requested', 'starttime': '2019-05-04T13:56:58.276068', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190504135642'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-05-04T13:56:58.276068', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190504135642'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-05-04T13:56:58.276068', 'endtime': None, 'percent_complete': 10, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190504135642'], 'mod

In [104]:
#Look up job status
ds.lookup_job_status_name(analyze_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-05-04T13:56:58.276068',
  'endtime': '2019-05-04T13:57:49.805',
  'percent_complete': 100,
  'job_type': 'AnalyzeData',
  'loss': None,
  'generations': None,
  'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'],
  'artifact_names': ['Darwin_analyze_data_artifact-20190504135642'],
  'model_name': None,
  'job_error': ''})

In [105]:
# Clean dataset
status, job_id = ds.clean_data(dataset_name=TRAIN_DATASET)
if not status:
    print(job_id)
else:
    print('Data has been cleaned!')

Data has been cleaned!


In [106]:
# Build unsupervised model
# 3 clusters because we are trying to predict the three overarching positions that all players fall into
model = "model" + "-" + TS
max_epochs = 20
n_clusters = 3
#n_clusters = 'auto'
status, job_id = ds.create_model(dataset_names=TRAIN_DATASET,
                                 model_name=model,
                                 max_epochs=max_epochs,
                                 n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Running', 'starttime': '2019-05-04T13:58:12.809662', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-05-04T13:58:12.809662', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-05-04T13:58:12.809662', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-05-04T13:58:12.8096

In [107]:
# look up job status
ds.lookup_job_status_name(job_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-05-04T13:58:12.809662',
  'endtime': '2019-05-04T13:58:53.164855',
  'percent_complete': 100,
  'job_type': 'TrainModel',
  'loss': None,
  'generations': 0,
  'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'],
  'artifact_names': None,
  'model_name': 'model-20190504135642',
  'job_error': ''})

In [108]:
# look up the model
ds.lookup_model_name(job_id['model_name'])

(True,
 {'type': 'Unsupervised',
  'updated_at': '2019-05-04T13:58:53.156516',
  'trained_on': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'],
  'loss': None,
  'generations': 0,
  'parameters': {'n_clusters': 3,
   'max_generation': 20,
   'train_time': '00:10',
   'recurrent': None,
   'max_unique_values': 50,
   'max_int_uniques': 15,
   'impute': 'mean',
   'big_data': False},
  'description': {'model': "UnsupervisedPipeline(anomaly=False, anomaly_prior=0.0015, auto_save_per=10,\n           clustering=True, clustermethod='GaussianMixture',\n           job_id='91002218-6e9e-11e9-a3d0-533194416626',\n           max_generation=20, max_time=600,\n           model_file='models/8f7f1eea-4fc6-11e9-ba76-eb31f920f59e_model-20190504135642',\n           n_clusters=3, preproc_anomaly=None, recurrent=None, verbose=2)",
   'genome_type': 'Unsupervised'},
  'train_time_seconds': 40,
  'algorithm': None,
  'running_job_id': None})

In [109]:
# Train some more
extra_epochs = 10
status, job_id = ds.resume_training_model(dataset_names=TRAIN_DATASET,
                                          model_name=model,
                                          max_epochs=extra_epochs,
                                          n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Running', 'starttime': '2019-05-04T13:59:24.229228', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-05-04T13:59:24.229228', 'endtime': '2019-05-04T13:59:33.239518', 'percent_complete': 100, 'job_type': 'UpdateModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190504135642', 'job_error': ''}


In [110]:
# Test model
# using this for predictions in the following cells
status, artifact = ds.run_model(TRAIN_DATASET, 
                                model, 
                                supervised=False)
sleep(1)
ds.wait_for_job(artifact['job_name'])

{'status': 'Running', 'starttime': '2019-05-04T13:59:41.449263', 'endtime': None, 'percent_complete': 0, 'job_type': 'RunModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['b7ab5d3487804dd291d937e578903cdc'], 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-05-04T13:59:41.449263', 'endtime': '2019-05-04T13:59:45.429971', 'percent_complete': 100, 'job_type': 'RunModel', 'loss': None, 'generations': 0, 'dataset_names': ['12-16_Combines_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['b7ab5d3487804dd291d937e578903cdc'], 'model_name': 'model-20190504135642', 'job_error': ''}


(True, 'Job completed')

In [111]:
# retrieve predictions
status, pred_file = ds.download_artifact(artifact['artifact_name'])

In [112]:
# show prediction
df = pd.read_csv(pred_file['filename'])
df.head()

,anomaly_score,predict_proba,prediction
0,-10.353864,"[4.592559076229764e-166, 1.0, 0.0]",1
1,-10.353864,"[1.36527841686267e-165, 1.0, 0.0]",1
2,-10.353864,"[7.35035621131701e-164, 1.0, 0.0]",1
3,-10.353864,"[7.070859733925672e-173, 1.0, 0.0]",1
4,-7.230650,"[1.6012689517045818e-178, 1.0, 0.0]",1


In [113]:
#Analyze model to retrieve feature importance 
#This will let us know which attributes matter the most when classifying players into our three positions
status, analyze_id = ds.analyze_model(job_id['model_name'], 
                                      job_name='Darwin_analyze_model_job-' + TS, 
                                      artifact_name='Darwin_analyze_model_artifact-' + TS)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_model_job-' + TS)
else:
    print(analyze_id)

{'status': 'Running', 'starttime': '2019-05-04T14:00:18.11042', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Darwin_analyze_model_artifact-20190504135642'], 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-05-04T14:00:18.11042', 'endtime': '2019-05-04T14:00:22.983206', 'percent_complete': 100, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Darwin_analyze_model_artifact-20190504135642'], 'model_name': 'model-20190504135642', 'job_error': ''}


In [114]:
#Get job status
ds.lookup_job_status_name('Darwin_analyze_model_job-' + TS)

(True,
 {'status': 'Complete',
  'starttime': '2019-05-04T14:00:18.11042',
  'endtime': '2019-05-04T14:00:22.983206',
  'percent_complete': 100,
  'job_type': 'AnalyzeModel',
  'loss': None,
  'generations': 0,
  'dataset_names': None,
  'artifact_names': ['Darwin_analyze_model_artifact-20190504135642'],
  'model_name': 'model-20190504135642',
  'job_error': ''})

In [115]:
#Print top 15 features
status, feature_importance = ds.download_artifact('Darwin_analyze_model_artifact-' + TS)
feature_importance[:15]

Pos = PG    0.150547
AST         0.121729
Pos = PF    0.087533
ORB         0.070452
Pos = SG    0.060073
TRB         0.051475
BLK         0.046046
_3PA        0.039042
Pos = SF    0.030724
DRB         0.030480
FG_         0.027077
TOV         0.023581
_3P         0.023414
PF          0.016010
_2P_        0.014432
dtype: float64

In [116]:
# Now we are using out prediction dataset (data from the 16/17 season)
#Since we can not analyze_predictions on datasets with more than 500 rows, we are only predicting 
#the 500 players with the most games

status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, PREDICT_DATASET))
if not status:
    print(dataset)
    
if status:
    dataset_by_row=dataset['dataset_name']
else:
    print("Upload data failed!")

In [117]:
#Run analyze predictions
status, analyze_id = ds.analyze_predictions(job_id['model_name'], 
                                            PREDICT_DATASET, 
                                            job_name='Analyze_prediction_job-' + TS, 
                                            artifact_name='Analyze_prediction_artifact-' + TS)
sleep(1)
if status:
    ds.wait_for_job('Analyze_prediction_job-' + TS)
else:
    print(analyze_id)

{'status': 'Running', 'starttime': '2019-05-04T14:00:50.481528', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzePredictions', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Analyze_prediction_artifact-20190504135642'], 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-05-04T14:00:50.481528', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzePredictions', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Analyze_prediction_artifact-20190504135642'], 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-05-04T14:00:50.481528', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzePredictions', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Analyze_prediction_artifact-20190504135642'], 'model_name': 'model-20190504135642', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-05-04T14:00:50

In [118]:
#Get job status
ds.lookup_job_status_name('Analyze_prediction_job-' + TS)

(True,
 {'status': 'Complete',
  'starttime': '2019-05-04T14:00:50.481528',
  'endtime': '2019-05-04T14:03:27.984098',
  'percent_complete': 100,
  'job_type': 'AnalyzePredictions',
  'loss': None,
  'generations': 0,
  'dataset_names': None,
  'artifact_names': ['Analyze_prediction_artifact-20190504135642'],
  'model_name': 'model-20190504135642',
  'job_error': ''})

In [119]:
#Print top 15 features
status, feature_importance = ds.download_artifact('Analyze_prediction_artifact-' + TS)
feature_importance.head()

,Rk_shap,Age_shap,G_shap,GS_shap,MP_shap,FG_shap,FGA_shap,FG__shap,_3P_shap,_3PA_shap,...,Tm = POR_shap,Tm = SAC_shap,Tm = SAS_shap,Tm = TOR_shap,Tm = TOT_shap,Tm = UTA_shap,Tm = WAS_shap,base_value,predicted_proba,predicted_class
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0.020416,0.021505,-0.063396,-0.04627,-0.038121,0.0,0.0,0.0,0.021175,0.018435,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
